In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-community transformers langchain-huggingface sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU pypdf langchain-text-splitters langchain-chroma rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 91.3 MB/s eta 0:00:

# Load API

In [ ]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Load Document

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/CC.pdf")
pages = loader.load()

In [ ]:
len(pages)

5

# Split Document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
len(docs)

14

# Load Embedding Model

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

# Store embeddings into Vector DB

In [ ]:
from langchain_chroma import Chroma

In [ ]:
# save to disk
db2 = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [ ]:
# load from disk
index = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Create Retriever

In [ ]:
# Semenatic Retriever
semantic_retriever = index.as_retriever(search_kwargs={"k": 3})

In [ ]:
# Keyword Retriever
from langchain_community.retrievers import BM25Retriever

In [ ]:
keyword_retriever = BM25Retriever.from_documents(docs)
keyword_retriever.k =  3

# Merge Retriever

In [ ]:
from langchain.retrievers import EnsembleRetriever

In [ ]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[semantic_retriever, keyword_retriever], weights=[0.5, 0.5]
)

# Call LLM

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = "HuggingFaceH4/zephyr-7b-alpha"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=1024,
    temperature=0.1,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Prompt

In [ ]:
## zephyr prompt

# template = """
# <|system|>
# You are a helpful AI Assistant that follows instructions extremely well.
# Use the following context to answer user question.

# Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

# CONTEXT: {context}
# </s>
# <|user|>
# {query}
# </s>
# <|assistant|>
# """

In [ ]:
template = """
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}

Query: {query}
"""

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)

# Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run Chain

In [ ]:
print(chain.invoke("Which bank did Tracy Q. Randall call?"))


Answer: Tracy Q. Randall called Bank of Wealth.


In [ ]:
print(chain.invoke("What is Tracy Q. Randall account number?"))


Answer: Tracy Q. Randall's account number is 805 -7845 -3895 -061.
